# Estimating Stop Frequency

This notebook illustrates how to re-estimate a single model component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch  # !conda install larch -c conda-forge # for estimation
import pandas as pd

We'll work in our `test` directory, where ActivitySim has saved the estimation data bundles.

In [2]:
os.chdir('C:\ABM3_dev\outputs')

# Load data and prep model for estimation

In [3]:
modelname = "stop_frequency"

from activitysim.estimation.larch import component_model
model, data = component_model(modelname, return_data=True)

c:\abm3_dev\activitysim\activitysim\estimation\larch\stop_frequency.py:114: DtypeWarning: Columns (34,35,48) have mixed types. Specify dtype option on import or set low_memory=False.
  chooser_data = pd.read_csv(
c:\abm3_dev\activitysim\activitysim\estimation\larch\stop_frequency.py:114: DtypeWarning: Columns (57) have mixed types. Specify dtype option on import or set low_memory=False.
  chooser_data = pd.read_csv(
c:\abm3_dev\activitysim\activitysim\estimation\larch\stop_frequency.py:114: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  chooser_data = pd.read_csv(
c:\abm3_dev\activitysim\activitysim\estimation\larch\general.py:324: UserWarning: coefficient dataframe missing 'constrain' column, setting all to 'F'
  warnings.warn(


# Review data loaded from the EDB

The next step is to read the EDB, including the coefficients, model settings, utilities specification, and chooser and alternative data.

In [4]:
spec_segments = [i.primary_purpose for i in data.settings.SPEC_SEGMENTS]
spec_segments

['work',
 'school',
 'univ',
 'social',
 'shopping',
 'eatout',
 'escort',
 'othmaint',
 'othdiscr',
 'atwork']

## Coefficients

There is one meta-coefficients dataframe for this component, which contains
parameters for all the matching coefficients in the various segmented 
files. When different segments have the same named coefficient with the same
value, it is assumed they should be estimated jointly.  If they have the same name
but different values in the coefficient files, then they are re-estimated
independently.

In [5]:
data.coefficients

,value,constrain
coefficient_name,,
coef_constants_for_outbound_trips_1out0in,-1.748000,F
coef_constants_for_outbound_trips_1out1in,-1.748000,F
coef_constants_for_outbound_trips_1out2in,-1.748000,F
coef_constants_for_outbound_trips_1out3in,-1.748000,F
coef_constants_for_outbound_trips_2out0in,-3.979000,F
...,...,...
coef_abm2_calibration_alternative_specific_constant_adjustment_1out0in_atwork,0.462488,F
coef_abm2_calibration_alternative_specific_constant_adjustment_1out1in_atwork,-0.576300,F
coef_abm2_calibration_alternative_specific_constant_adjustment_1out2in_atwork,-1.384033,F


## Utility specification

The utility spec files are unique to each segment model.  The estimation mode larch pre-processor
for the stop frequency model modifies the spec files to account for jointly re-estimated
parameters.

In [6]:
# data.spec[0]

## Chooser data

The chooser data is unique to each segment model. 

In [7]:
# data.chooser_data[0]

# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [8]:
data.chooser_data[0].primary_purpose.value_counts()

work    9535
Name: primary_purpose, dtype: int64

In [9]:
for i in range(0, len(data.chooser_data)):
    print(f"model {i} is purpose {data.chooser_data[i].primary_purpose.value_counts().idxmax()}")

model 0 is purpose work
model 1 is purpose school
model 2 is purpose univ
model 3 is purpose social
model 4 is purpose shopping
model 5 is purpose eatout
model 6 is purpose escort
model 7 is purpose othmaint
model 8 is purpose othdiscr
model 9 is purpose atwork


In [12]:
# model[0].estimate(method='BHHH') # can also estimate models one at a time
model.estimate(method='SLSQP', options={"maxiter": 1000})

req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided
req_data does not request avail_ca or avail_co but it is set and being provided


,value,initvalue,nullvalue,minimum,maximum,holdfast,note,std_err,t_stat,robust_std_err,robust_t_stat,unconstrained_std_err,unconstrained_t_stat,likelihood_ratio,best
coef_abm2_calibration__alternative_specific_constant_adjustment_0out1in,1.884660e+03,0.224980,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_abm2_calibration__alternative_specific_constant_adjustment_0out2in,-4.798115e+03,0.731748,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_abm2_calibration__alternative_specific_constant_adjustment_0out3in,1.852926e+03,1.537128,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_abm2_calibration__alternative_specific_constant_adjustment_1out0in,-5.712700e+03,-0.281447,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_abm2_calibration__alternative_specific_constant_adjustment_1out1in,8.818440e+03,-0.772998,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
coef_tour_duration_in_hours_2out3in_atwork,7.635355e+06,0.471195,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_tour_duration_in_hours_3out0in_atwork,-4.260076e+03,0.471195,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_tour_duration_in_hours_3out1in_atwork,4.491385e+06,0.471195,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
coef_tour_duration_in_hours_3out2in_atwork,6.737082e+05,0.471195,0.0,NaN,NaN,0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


: 

: 

### Estimated coefficients

In [1]:
# model.parameter_summary()

# Output Estimation Results

The stop frequency model include seperate coefficient file for each segment,
and has a special writer method to seperate the coefficient by segment
after estimation.

In [ ]:
from activitysim.estimation.larch.stop_frequency import update_segment_coefficients
result_dir = data.edb_directory/"estimated"
update_segment_coefficients(
    model, data, result_dir,
    output_file="stop_frequency_coefficients_{segment_name}_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [ ]:
for m, segment in zip(model, data.segments):
    m.to_xlsx(
        result_dir/f"{modelname}_{segment}_model_estimation.xlsx", 
        data_statistics=False,
    )

# Next Steps

The final step is to either manually or automatically copy the `stop_frequency_coefficients_*_revised.csv` files to the configs folder, rename them to `stop_frequency_coefficients_*.csv`, and run ActivitySim in simulation mode.

In [ ]:
pd.read_csv(result_dir/"stop_frequency_coefficients_work_revised.csv")